In [1]:
# bayesian optimization 설명
# http://research.sualab.com/introduction/practice/2019/02/19/bayesian-optimization-overview-1.html

# parameter 설명
# http://machinelearningkorea.com/2019/09/29/lightgbm-%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0/

# 자료가 좀 있는듯
# http://machinelearningkorea.com/

## classification?
# https://data-newbie.tistory.com/160
# https://www.kaggle.com/sz8416/simple-bayesian-optimization-for-lightgbm

# regression
# https://www.kaggle.com/chocozzz/bayesian-optimization-for-lightgbm

In [12]:
import numpy as np
import pandas as pd
import math

import seaborn as sns
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from scipy.stats import chi2_contingency
from lightgbm import LGBMRegressor
from sklearn.preprocessing import StandardScaler
from bayes_opt import BayesianOptimization

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [3]:
train = pd.read_csv('../data/train.csv', index_col='id')
test = pd.read_csv('../data/test.csv', index_col='id')
submission = pd.read_csv('../data/sample_submission.csv', index_col='id')

In [5]:
features = ['X00', 'X01', 'X02', 'X03', 'X05', 'X06', 'X07', 'X08', 'X09', 'X10', 'X11', 'X12', 'X13', 'X15', 'X17', 'X18', 'X20', 'X21', 'X22', 'X23', 'X24', 'X25', 'X26', 'X27', 'X28', 'X29', 'X30', 'X31', 'X32', 'X33', 'X34', 'X35', 'X37', 'X38', 'X39']

trainNotNull = train[train['Y18'].isnull()]

## perm 작업을 하기전에 model fit을 수행함
X = train[features]
y = train['Y00']

In [7]:
import lightgbm as lgb

params = {
            "objective" : "regression", "bagging_fraction" : 0.8, "bagging_freq": 1,
            "min_child_samples": 20, "reg_alpha": 1, "reg_lambda": 1,"boosting": "rf",
            "learning_rate" : 0.01, "subsample" : 0.8, "colsample_bytree" : 0.8, "verbosity": -1, "metric" : 'rmse'
        }
train_data = lgb.Dataset(data=X, label=y, categorical_feature = list(X.columns),free_raw_data=False)

# Perform the cross-validation with given paramaters 
# https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.cv.html    
cv_result = lgb.cv(params, train_data, nfold=5, seed=0, verbose_eval =200,stratified=False,shuffle=False)

In [8]:
-1.0 * np.array(cv_result['rmse-mean'])

array([-7.33838456, -6.82699482, -6.64162526, -6.56420956, -6.56673587,
       -6.55618738, -6.58263813, -6.59759106, -6.62323799, -6.60574565,
       -6.60948086, -6.61229932, -6.61602489, -6.64352393, -6.64492704,
       -6.63830164, -6.62538523, -6.62773945, -6.66691257, -6.65261558,
       -6.66380831, -6.66725532, -6.66514802, -6.6388137 , -6.66013809,
       -6.65131096, -6.64865323, -6.64843859, -6.66453943, -6.65139525,
       -6.66665071, -6.65694915, -6.65781379, -6.64491259, -6.64414746,
       -6.64625003, -6.64625099, -6.64761692, -6.63896814, -6.64898876,
       -6.65836137, -6.65344548, -6.65803273, -6.6559006 , -6.6573963 ,
       -6.65696213, -6.65866507, -6.65473171, -6.64281877, -6.64690239,
       -6.64958462, -6.64745505, -6.65406146, -6.64683628, -6.65005272,
       -6.64654234, -6.6459762 , -6.64407586, -6.64050893, -6.64014744,
       -6.64167562, -6.64430823, -6.64419944, -6.64476765, -6.64242197,
       -6.64432206, -6.64248166, -6.64532249, -6.64664521, -6.64

In [9]:
(-1.0 * np.array(cv_result['rmse-mean'])).max()

-6.55618737764928

In [10]:
def lgb_eval(num_leaves, feature_fraction, max_depth , min_split_gain, min_child_weight):
    params = {
            "objective" : "regression", "bagging_fraction" : 0.8, "bagging_freq": 1,
            "min_child_samples": 20, "reg_alpha": 1, "reg_lambda": 1,"boosting": "rf",
            "learning_rate" : 0.01, "subsample" : 0.8, "colsample_bytree" : 0.8, "verbosity": -1, "metric" : 'rmse'
        }
    params['feature_fraction'] = max(min(feature_fraction, 1), 0)
    params['max_depth'] = int(round(max_depth))
    params['num_leaves'] = int(round(num_leaves))
    params['min_split_gain'] = min_split_gain
    params['min_child_weight'] = min_child_weight
    cv_result = lgb.cv(params, train_data, nfold=5, seed=0, verbose_eval =200,stratified=False)
    return (-1.0 * np.array(cv_result['rmse-mean'])).max()

In [13]:
lgbBO = BayesianOptimization(lgb_eval, {'feature_fraction': (0.1, 0.9),
                                            'max_depth': (5, 9),
                                            'num_leaves' : (200,300),
                                            'min_split_gain': (0.001, 0.1),
                                            'min_child_weight': (5, 50)}, random_state=0)

In [14]:
lgbBO.maximize(init_points=5, n_iter=5,acq='ei')

|   iter    |  target   | featur... | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------
|  1        | -4.534    |  0.5391   |  7.861    |  32.12    |  0.05494  |  242.4    |
|  2        | -4.49     |  0.6167   |  6.75     |  45.13    |  0.0964   |  238.3    |
|  3        | -4.487    |  0.7334   |  7.116    |  30.56    |  0.09263  |  207.1    |
|  4        | -5.099    |  0.1697   |  5.081    |  42.47    |  0.07804  |  287.0    |
|  5        | -4.504    |  0.8829   |  8.197    |  25.77    |  0.07827  |  211.8    |
|  6        | -5.499    |  0.1      |  9.0      |  50.0     |  0.001    |  209.7    |
|  7        | -4.608    |  0.3836   |  6.472    |  5.099    |  0.04567  |  200.1    |
|  8        | -4.506    |  0.9      |  9.0      |  5.0      |  0.001    |  300.0    |
|  9        | -4.517    |  0.9      |  5.0      |  5.0      |  0.1      |  267.6    |
|  10       | -4.517    |  0.9      |  5.0      |  23.

In [15]:
def bayes_parameter_opt_lgb(X, y, init_round=15, opt_round=25, n_folds=5, random_seed=0, n_estimators=10000, learning_rate=0.05, output_process=False):
    # prepare data
    train_data = lgb.Dataset(data=X, label=y, categorical_feature = list(X.columns),free_raw_data=False)
    # parameters

    def lgb_eval(num_leaves, feature_fraction, max_depth , min_split_gain, min_child_weight):
        params = {
            "objective" : "regression", "bagging_fraction" : 0.8, "bagging_freq": 1,
            "min_child_samples": 20, "reg_alpha": 1, "reg_lambda": 1,"boosting": "rf",
            "learning_rate" : 0.01, "subsample" : 0.8, "colsample_bytree" : 0.8, "verbosity": -1, "metric" : 'rmse'
        }
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['max_depth'] = int(round(max_depth))
        params['num_leaves'] = int(round(num_leaves))
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight

        cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, verbose_eval =200,stratified=False)
        return (-1.0 * np.array(cv_result['rmse-mean'])).max()
    
        # range 
    lgbBO = BayesianOptimization(lgb_eval, {'feature_fraction': (0.1, 0.9),
                                            'max_depth': (5, 9),
                                            'num_leaves' : (200,300),
                                            'min_split_gain': (0.001, 0.1),
                                            'min_child_weight': (5, 50)}, random_state=0)
        # optimize
    lgbBO.maximize(init_points=init_round, n_iter=opt_round,acq='ei')

        # output optimization process
    if output_process==True: lgbBO.points_to_csv("bayes_opt_result.csv")

        # return best parameters
    # return lgbBO.res['max']['max_params']
    return lgbBO

opt_params = bayes_parameter_opt_lgb(X, y, init_round=10, opt_round=10, n_folds=5, random_seed=0, n_estimators=1000, learning_rate=0.01)

|   iter    |  target   | featur... | max_depth | min_ch... | min_sp... | num_le... |
-------------------------------------------------------------------------------------
|  1        | -4.534    |  0.5391   |  7.861    |  32.12    |  0.05494  |  242.4    |
|  2        | -4.49     |  0.6167   |  6.75     |  45.13    |  0.0964   |  238.3    |
|  3        | -4.487    |  0.7334   |  7.116    |  30.56    |  0.09263  |  207.1    |
|  4        | -5.099    |  0.1697   |  5.081    |  42.47    |  0.07804  |  287.0    |
|  5        | -4.504    |  0.8829   |  8.197    |  25.77    |  0.07827  |  211.8    |
|  6        | -4.5      |  0.6119   |  5.573    |  47.51    |  0.05266  |  241.5    |
|  7        | -4.629    |  0.3116   |  8.097    |  25.53    |  0.05727  |  201.9    |
|  8        | -4.499    |  0.5941   |  7.448    |  32.76    |  0.09443  |  268.2    |
|  9        | -4.602    |  0.3876   |  6.748    |  36.39    |  0.006962 |  266.7    |
|  10       | -4.491    |  0.6365   |  5.842    |  10.

In [19]:
opt_params.max['params']
# opt_params.res['max']['max_params']

{'feature_fraction': 0.7333800304661316,
 'max_depth': 7.115579679011618,
 'min_child_weight': 30.562005249226953,
 'min_split_gain': 0.09263406719097345,
 'num_leaves': 207.1036058197887}

In [21]:
lgb_params = {
    'feature_fraction': 0.7333800304661316,
    'max_depth': 7,
    'min_child_weight': 30.562005249226953,
    'min_split_gain': 0.09263406719097345,
    'num_leaves': 207
}

In [ ]:
x_train, x_val, y_train, y_val = train_test_split(X, y, random_state=0)


clf = lgb.train(lgb_params, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds = 100)
# model.fit(x_train, y_train, eval_set=[(x_val, y_val)], early_stopping_rounds=100, verbose=False)